In [1]:
import sys, os
import ipywidgets as widgets
from IPython.display import display, clear_output

base = r"C:\Users\camil\OneDrive\Bureau\house_prediction"
sys.path.append(os.path.join(base, "src", "interface"))

from predict_price import PricePredictor

pp = PricePredictor()

print("Loaded states:", len(pp.states))
print("Loaded cities:", len(pp.cities))

print("Test encode:", pp.encode_state("California"))
print("City encode:", pp.encode_city("Los Angeles"))

# Example ZIP codes
print("ZIP codes:", pp.possible_zip_codes("California", "Los Angeles")[:10])


Loaded states: 54
Loaded cities: 16934
Test encode: 4
City encode: 1359929.998874666
ZIP codes: [90001, 90002, 90003, 90004, 90005, 90006, 90007, 90008, 90010, 90011]


In [ ]:
import sys, os
import ipywidgets as widgets
from IPython.display import display, clear_output

base = r"C:\Users\camil\OneDrive\Bureau\house_prediction"
sys.path.append(os.path.join(base, "src", "interface"))

from predict_price import PricePredictor

pp = PricePredictor()

print(f"Dataset loaded: {len(pp.df)} rows")
print(f"States: {len(pp.states)}")

state_dropdown = widgets.Dropdown(
    options=pp.states,
    description="State:",
    layout=widgets.Layout(width="300px")
)

city_dropdown = widgets.Dropdown(
    options=["Select state"],
    description="City:",
    layout=widgets.Layout(width="300px")
)

zip_dropdown = widgets.Dropdown(
    options=["Select city"],
    description="ZIP:",
    layout=widgets.Layout(width="300px")
)

bed_input = widgets.BoundedFloatText(value=3, min=0, max=20, description="Bedrooms:")
bath_input = widgets.BoundedFloatText(value=2, min=0, max=20, description="Bathrooms:")
acre_input = widgets.BoundedFloatText(value=0.20, min=0, max=50, description="Acre lot:")
size_input = widgets.BoundedFloatText(value=1500, min=100, max=15000, description="House size:")

predict_button = widgets.Button(description="Predict price", button_style="success")
output = widgets.Output()

def on_state_change(change):
    state = state_dropdown.value
    cities = pp.cities_for_state(state)
    if cities:
        city_dropdown.options = cities
        city_dropdown.value = cities[0]
    else:
        city_dropdown.options = ["No cities"]
        zip_dropdown.options = ["No ZIP"]

def on_city_change(change):
    state = state_dropdown.value
    city = city_dropdown.value
    zips = pp.zips_for_state_city(state, city)
    if zips:
        zip_dropdown.options = zips
        zip_dropdown.value = zips[0]
    else:
        zip_dropdown.options = ["No ZIP"]

def on_predict(b):
    with output:
        clear_output()
        if "No ZIP" in str(zip_dropdown.value):
            print("Please select a valid ZIP.")
            return
        
        price, accuracy = pp.predict(
            bed_input.value,
            bath_input.value,
            acre_input.value,
            city_dropdown.value,
            state_dropdown.value,
            int(zip_dropdown.value),
            size_input.value
        )
        
        print()
        print(f"Predicted price: ${price:,.0f}")
        print(f"Model precision: {accuracy:.2f}%")   # <-- ICI LA LIGNE IMPORTANTE !



state_dropdown.observe(on_state_change, names="value")
city_dropdown.observe(on_city_change, names="value")
predict_button.on_click(on_predict)

on_state_change(None)

ui = widgets.VBox([
    widgets.HTML("<h2>House price predictor</h2>"),
    state_dropdown,
    city_dropdown,
    zip_dropdown,
    bed_input,
    bath_input,
    acre_input,
    size_input,
    predict_button,
    output
])

display(ui)


Dataset loaded: 1341789 rows
States: 54
